In [ ]:
!export CUDA_VISIBLE_DEVICES=4

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
def sound( x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    if label is '':
        display( Audio( x, rate=rate))
    else:
        display( HTML( 
        '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
        '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
        ))
import pdb

In [ ]:
emb_hyp = {
    'audio/sample_rate': 8000,
    'embedder/n_fft': 512,
    'embedder/hop': 160,
    'embedder/n_mels': 40,
    'embedder/num_layers': 3,
    'embedder/lstm_hidden': 768,
    'embedder/emb_dim': 256,
    'embedder/pretrained_path': '/mnt/data/zhepei/outputs/tse_semi/spid/vfpt_spid/2021-08-29-22-19-31_blstm_softmax_8k_bs46_4sec/chkpt/chkpt_100000.pt',
    'embedder/trainable': False,
}

from spid_modules.embedder import Embedder
embedder = Embedder(emb_hyp)
embedder.eval()

In [ ]:
hyp = {
    'sample_rate': 8000,
    'training_signal_len': 40000,
    'train_dataloader_opts': {
        'batch_size': 1,
        'num_workers': 0,
    },
    'valid_dataloader_opts': {
        'batch_size': 1,
        'num_workers': 0,
    },
    'data_folder': '/mnt/data/wham/wham_original',
    'wsj_folder': '/mnt/data/wsj0.8k',
    'base_folder_dm_info_list': [
        {
            'path': '/mnt/data/wsj0.8k/si_tr_s/',
            'ext': 'wav',
            'type': 'clean',
        }
    ],
    'data_clean_prob': 1.,
    
#     'train_txtpath': '/mnt/data/Speech/wsj_tse/mix_2_spk_tr_extr.txt',
#     'train_wham_folder': '/mnt/data/wham/wham_original/wav8k/min/tr',
    'train_data': '/mnt/data/zhepei/outputs/sb_tse/results/2021-10-11+21-30-05+seed_1234+xformer-wham/save/wham_tse_tr.csv',
    'valid_data': '/mnt/data/zhepei/outputs/sb_tse/results/2021-10-27+03-53-37+seed_1234+xformer-wham-pre/save/wham_tse_tt.csv',

}

In [ ]:
import torch
import numpy as np
from speechbrain.dataio.batch import PaddedBatch

# Test Dynamic mixing

In [ ]:
from data.wham_data_utils import dynamic_mixing_prep
train_dl = dynamic_mixing_prep(hyp, 'train')

In [ ]:
def listen_batch(batch):
    mix = batch['mix_sig'].data
    s1 = batch['s1_sig'].data
    s2 = batch['s2_sig'].data
    enr = batch['enr_sig'].data
    noise = batch['noise_sig'].data
    diff = mix - s1 - s2 - noise
    print(abs(mix).max())
    print((diff**2).mean())
    sound(mix[0].numpy(), rate=hyp['sample_rate'], label='mix')
    sound(s1[0].numpy(), rate=hyp['sample_rate'], label='s1')
    sound(s2[0].numpy(), rate=hyp['sample_rate'], label='s2')
    sound(enr[0].numpy(), rate=hyp['sample_rate'], label='enr')

In [ ]:
def compute_cos_sim(v1, v2):
    return (v1 * v2).sum() / (torch.norm(v1) * torch.norm(v2))

def check_emb(batch):
    s1 = batch['s1_sig'].data
    s2 = batch['s2_sig'].data
    enr = batch['enr_sig'].data
    s1_emb = embedder(s1)
    s2_emb = embedder(s2)
    enr_emb = embedder(enr)
    pos_sim = compute_cos_sim(s1_emb[0], enr_emb[0]).item()
    neg_sim = compute_cos_sim(s2_emb[0], enr_emb[0]).item()
    print('Positive sim: {} -- Negative sim: {} -- diff: {}'.format(pos_sim, neg_sim, pos_sim-neg_sim))

def check_emb_noise(batch):
    s1 = batch['s1_sig'].data
    s2 = batch['s2_sig'].data
    enr = batch['enr_sig'].data
    noise = batch['noise_sig'].data
    s1_emb = embedder(s1+noise)
    s2_emb = embedder(s2+noise)
    min_len = min(enr.shape[-1], noise.shape[-1])
    enr_emb = embedder(enr[..., :min_len]+noise[..., :min_len])
    pos_sim = compute_cos_sim(s1_emb[0], enr_emb[0]).item()
    neg_sim = compute_cos_sim(s2_emb[0], enr_emb[0]).item()
    print('Positive sim: {} -- Negative sim: {} -- diff: {}'.format(pos_sim, neg_sim, pos_sim-neg_sim))

In [ ]:
for i, batch in enumerate(train_dl):  
    if i == 1:
        listen_batch(batch)
        break

In [ ]:
for i, batch in enumerate(train_dl):  
    check_emb(batch)
    if i == 20:
        break

# Test static mixing 

In [ ]:
from data.wham_data_utils import static_data_prep
valid_ds = static_data_prep(hyp, 'valid')
valid_dl = torch.utils.data.DataLoader(
        valid_ds,
        batch_size=hyp["valid_dataloader_opts"]["batch_size"],
        num_workers=hyp["valid_dataloader_opts"]["num_workers"],
        collate_fn=PaddedBatch,
        worker_init_fn=lambda x: np.random.seed(
            int.from_bytes(os.urandom(4), "little") + x
        ),
    )

In [ ]:
for i, batch in enumerate(valid_dl):  
    if i == 1:
        listen_batch(batch)
        break

In [ ]:
for i, batch in enumerate(valid_dl):  
    check_emb(batch)
    if i == 20:
        break

In [ ]:
for i, batch in enumerate(valid_dl):  
    check_emb(batch)
    if i == 20:
        break

In [ ]:
for i, batch in enumerate(valid_dl):  
    check_emb_noise(batch)
    if i == 20:
        break

In [ ]:
for i, batch in enumerate(valid_dl):  
    check_emb_noise(batch)
    if i == 20:
        break